In [3]:
import os
os.chdir("../")

In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_files(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents=loader.load()
    return documents

In [7]:
extracted_data=load_pdf_files("data")

In [10]:
from typing import List
from langchain.schema import Document


def filter_to_min(docs: List[Document])->List[Document]:
    minimal_docs: List[Document]=[]
    for doc in docs:
        src=doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )
    return minimal_docs

In [12]:
minimal_docs=filter_to_min(extracted_data)

In [14]:
#split docs in chunks 
def split_into_chunks(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    ) 
    texts_chunk=text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [16]:
texts_chunk=split_into_chunks(minimal_docs)
print(f"number of chunks: {len(texts_chunk)}")

number of chunks: 637


In [17]:
from langchain.embeddings import HuggingFaceEmbeddings


def download_embeddings():
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(
        model_name=model_name,
    )
    return embeddings

embedding=download_embeddings()

C:\Users\hp\AppData\Local\Temp\ipykernel_21856\2539509177.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
vector=embedding.embed_query("hello hamd")
vector

In [20]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [21]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [22]:
from pinecone import Pinecone

pinecone_api_key=PINECONE_API_KEY

pc=Pinecone(api_key=pinecone_api_key)

In [ ]:
from pinecone import ServerlessSpec

index_name="Hearable_chatbot"
